In [1]:
%load_ext autoreload
%autoreload 2
%load_ext memory_profiler

In [2]:
import os
from tempfile import mkdtemp
import numpy as np
import matplotlib.pyplot as plt

from ics.hxutils import hxramp

from h4rg_analysis import ramputils,io

import sys
sys.path.append('../scripts/')
import do_simpleutr

plt.rcParams['figure.figsize'] = (8,8)
bestbaselines = open ('../data/paths/best_ever_baseline_darks_8_21_2022.txt', 'r').read().splitlines()[1:]

In [3]:
pt = io.jhu2tiger ( bestbaselines[1] )
tag = os.path.basename(pt).strip('.fits')

In [4]:
%%memit -r1
ramp = hxramp.HxRamp( pt )

peak memory: 85.66 MiB, increment: 0.08 MiB


In [6]:
r0=0
r1=-1
tot_ref = io.irpStack_memmap (ramp, r0=r0, r1=r1)

In [24]:
i = 3
step = 512

In [25]:
%%memit -r1
ref = tot_ref[:,step*i:step*(i+1),step*i:step*(i+1)]
process = psutil.Process(os.getpid())
print(process.memory_info().rss/1e9)
var, params = do_simpleutr.utrstats ( ref )

10.173263872
peak memory: 11801.79 MiB, increment: 2099.81 MiB


In [21]:
%%memit -r1
ref = tot_ref[:,step*i:step*(i+1),step*i:step*(i+1)]
process = psutil.Process(os.getpid())
print(process.memory_info().rss/1e9)
var, params = do_simpleutr.utrstats ( ref )

10.171158528
peak memory: 11799.99 MiB, increment: 2100.02 MiB


In [13]:
import psutil

In [64]:
%%memit -r1
for r_i in np.arange(r0, r1+1):
    n = ramp._readIdxToFITSIdx ( r_i )
    extname = f'REF_{n}'
    irpImage = ramp.fits[extname].read ()
    irpImage = hxramp.constructFullIrp(irpImage, ramp.nchan, refPix=ramp.interleaveOffset)
    fp[r_i] = irpImage
    #fp.flush ()
    break


peak memory: 267.64 MiB, increment: 63.81 MiB


In [72]:
process = psutil.Process(os.getpid())
print(process.memory_info().rss/1e9)  # in bytes 

0.21372928
